### MDSI Advanced Machine Learning Applications

**Student: Ivan Cheung**

**Assignment: AT2**


In [1]:
## load libraries
import pandas as pd

In [2]:
## load training data

train_df = pd.read_parquet('../../data/interim/train_clean_3.parquet')

### Pre-processing:

1) Create preprocessing function:
- split up date value, to support regression modelling.

In [3]:
### test code
train_df

,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,event_name,event_type,sell_price
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,None,None,0.00
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,None,None,0.00
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,None,None,0.00
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,None,None,0.00
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,None,None,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
47107045,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1541,2,2015-04-18,11512,None,None,2.88
47107046,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,None,None,2.00
47107047,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,None,None,3.98
47107048,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,None,None,1.28


In [8]:
print("train_df['item_id'] " + str(train_df['item_id'].nunique()))
print("train_df['dept_id'] " + str(train_df['dept_id'].nunique()))
print("train_df['cat_id'] " + str(train_df['cat_id'].nunique()))
print("train_df['event_name'] " + str(train_df['event_name'].nunique()))
print("train_df['event_type'] " + str(train_df['event_type'].nunique()))
print("train_df['store_id'] " + str(train_df['store_id'].nunique()))
print("train_df['state_id'] " + str(train_df['state_id'].nunique()))

train_df['item_id'] 3049
train_df['dept_id'] 7
train_df['cat_id'] 3
train_df['event_name'] 30
train_df['event_type'] 4
train_df['store_id'] 10
train_df['state_id'] 3


In [9]:
## define label encode cols, OHE cols and num cols
num_cols = ['items_sold', 'sell_price']
label_cols = ['item_id', 'event_name']
cat_cols = ['dept_id', 'cat_id', 'event_type', 'state_id', 'store_id']